## **Import Dependencies**

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import rasterio as rio
import os
from scipy.ndimage import generic_filter
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = 'Times New Roman'

data_dir = r'E:\HALDER\DATA'

## **Read the Datasets**

In [3]:
lulc_path = os.path.join(data_dir, 'CLC_LULC_2021_DE', 'CLCplus_RASTER_2021_010m_03035', 
                         'CLCplus_RASTER_2021_010m_03035', 'CLCplus_RASTER_2021_010m_03035.tif')

with rio.open(lulc_path) as src:
    lulc = src.read(1)
    profile = src.profile

In [7]:
def shannon_entropy(window):
    """Compute SHDI for a flattened moving window."""
    values = window[window > 0]  # optionally ignore background/NoData
    if len(values) == 0:
        return 0
    counts = Counter(values)
    total = sum(counts.values())
    proportions = np.array([count / total for count in counts.values()])
    return -np.sum(proportions * np.log(proportions))

def compute_shdi_moving_window(input_raster, output_raster, window_size=3):
    with rio.open(input_raster) as src:
        data = src.read(1)
        profile = src.profile
        nodata = src.nodata or 0  # set default NoData if undefined

    # Replace nodata with 0 if needed (assuming 0 is not a valid class)
    data = np.where(data == nodata, 0, data)

    # Apply SHDI function over the raster using a moving window
    result = generic_filter(
        data,
        function=shannon_entropy,
        size=window_size,
        mode='constant',
        cval=0  # pad with 0s outside the edges
    )

    # Save result
    profile.update(dtype='float32', count=1, nodata=None)
    with rio.open(output_raster, 'w', **profile) as dst:
        dst.write(result.astype(np.float32), 1)

In [ ]:
compute_shdi_moving_window(lulc_path, 'shdi_window.tif', window_size=9)